<a href="https://colab.research.google.com/github/theonlyNischal/ML-DL-Projects-and-Paper-Implementation/blob/master/1_Classic_Neural_Networks_Implementation/1-Lenet-5-Pytorch-Improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Today, we will implement the classical LeNet-5 architecture in Pytorch. In LeNet-5, 5 means there are 5 learned layers, 2 of them are convolutional layer and remaining 3 are dense/linear layer.

We will give modern improvements to the classical architecture, such as using

- Using max-pooling or ReLUs instead of average pooling.
- Using softmax instead of RBF-based classification


https://d2l.ai/_images/lenet.svg

Fig: Architecture of LeNet-5. Input is a handwritten digit, the output is a probability over 10 possible outcomes.


In [1]:
class ClassicLeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(ClassicLenet5).__init__()

        # convolutional layers
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=6,
            kernel_size=5,
            padding=0
        )
        self.conv2 = nn.Conv2d(
            in_channels=6,
            out_channels=16,
            kernel_size=5
        )
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        # FC layers
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.tanh(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = torch.tanh(x)
        x = self.pool(x)

        # Flatten
        x = x.view(x.size(0), -1)


        # FC Layers
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)

        return x

NameError: name 'nn' is not defined